<a href="https://colab.research.google.com/github/SeanAntosiak/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module4-makefeatures/LS_DS_124_Make_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Make features

Objectives
-  understand the purpose of feature engineering
-  work with strings in pandas
- work with dates and times in pandas

Links
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series

## Get LendingClub data

[Source](https://www.lendingclub.com/info/download-data.action)

In [0]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-07-11 21:52:31--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [        <=>         ]  21.51M  1.66MB/s    in 13s     

2019-07-11 21:52:45 (1.64 MB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22559146]



In [0]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [0]:
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

## Load LendingClub data

pandas documentation
- [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
- [`options.display`](https://pandas.pydata.org/pandas-docs/stable/options.html#available-options)

## Work with strings

For machine learning, we usually want to replace strings with numbers.

We can get info about which columns have a datatype of "object" (strings)

### Convert `int_rate`

Define a function to remove percent signs from strings and convert to floats

Apply the function to the `int_rate` column

### Clean `emp_title`

Look at top 20 titles

How often is `emp_title` null?

Clean the title and handle missing values

### Create `emp_title_manager`

pandas documentation: [`str.contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html)

## Work with dates

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

# ASSIGNMENT

- Replicate the lesson code.

- Convert the `term` column from string to integer.

- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.

- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [7]:
!wget !wget https://resources.lendingclub.com/LoanStats_2018Q3.csv.zip 

--2019-07-11 21:54:15--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2019-07-11 21:54:15--  https://resources.lendingclub.com/LoanStats_2018Q3.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q3.csv.zip.3’

LoanStats_2018Q3.cs     [     <=>            ]  21.68M  1.67MB/s    in 14s     

2019-07-11 21:54:28 (1.60 MB/s) - ‘LoanStats_2018Q3.csv.zip.3’ saved [22734399]

FINISHED --2019-07-11 21:54:28--
Total wall clock time: 14s
Downloaded: 1 files, 22M in 14s (1.60 MB/s)


In [8]:
!unzip LoanStats_2018Q3.csv.zip

Archive:  LoanStats_2018Q3.csv.zip
replace LoanStats_2018Q3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [9]:
!head LoanStats_2018Q3.csv
print('')
print('break')
print('')
!tail LoanStats_2018Q3.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

In [10]:
df1 = pd.read_csv('LoanStats_2018Q3.csv', skiprows=[0], skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [11]:
df1.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,25000,25000,25000,36 months,16.14%,880.66,C,C4,Business Planner,10+ years,RENT,82000.0,Source Verified,Sep-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,940xx,CA,19.60,0,Mar-1994,1,NaN,NaN,10,0,15437,59.4%,17,w,19828.26,19828.26,7881.110000,7881.11,5171.74,2709.37,0.0,0.0,0.0,Jun-2019,880.66,Jul-2019,Jun-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,39316,2,2,1,1,4.0,23879,59.0,4,5,5932,59.0,26000,0,0,1,6,3932.0,1844.0,87.7,0,0,127.0,294,1,1,0,1.0,NaN,1.0,NaN,0,4,6,4,4,5,8,12,6,10,0.0,0,0,5,94.1,75.0,0,0,66646,39316,15000,40646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,20000,20000,20000,36 months,7.84%,625.26,A,A4,Teacher,10+ years,MORTGAGE,48186.0,Not Verified,Sep-2018,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,641xx,MO,24.68,0,Aug-1996,0,NaN,NaN,17,0,23822,58.8%,38,w,0.00,0.00,20784.182581,20784.18,20000.00,784.18,0.0,0.0,0.0,Apr-2019,17105.26,NaN,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,191246,0,9,0,2,16.0,14800,48.0,1,1,11800,54.0,40500,1,0,0,3,11250.0,13078.0,57.8,0,0,265.0,239,8,8,2,8.0,NaN,16.0,NaN,0,3,4,6,7,27,7,9,4,17,0.0,0,0,1,100.0,16.7,0,0,234182,38622,31000,30832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,15000,15000,15000,60 months,15.02%,357.01,C,C3,Salesperson,10+ years,RENT,70000.0,Not Verified,Sep-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,190xx,PA,18.05,0,Jul-1995,0,42.0,NaN,7,0,8112,66.5%,11,w,13398.12,13398.12,3188.060000,3188.06,1601.88,1586.18,0.0,0.0,0.0,Jun-2019,357.01,Jul-2019,Jun-2019,1,80.0,1,Individual,NaN,NaN,NaN,0,0,

In [12]:
print(df1.shape)

print('')

df1.isnull().sum().sort_values(ascending=False)

(128194, 144)



id                                            128194
desc                                          128194
member_id                                     128194
url                                           128194
orig_projected_additional_accrued_interest    128132
hardship_type                                 128127
hardship_last_payment_amount                  128127
hardship_payoff_balance_amount                128127
hardship_reason                               128127
hardship_loan_status                          128127
hardship_dpd                                  128127
hardship_length                               128127
payment_plan_start_date                       128127
hardship_end_date                             128127
hardship_start_date                           128127
hardship_amount                               128127
deferral_term                                 128127
hardship_status                               128127
settlement_percentage                         

In [13]:
df2 = df1.drop(['id', 'member_id', 'desc', 'url'], axis=1)
df2.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,25000,25000,25000,36 months,16.14%,880.66,C,C4,Business Planner,10+ years,RENT,82000.0,Source Verified,Sep-2018,Current,n,credit_card,Credit card refinancing,940xx,CA,19.60,0,Mar-1994,1,NaN,NaN,10,0,15437,59.4%,17,w,19828.26,19828.26,7881.110000,7881.11,5171.74,2709.37,0.0,0.0,0.0,Jun-2019,880.66,Jul-2019,Jun-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,39316,2,2,1,1,4.0,23879,59.0,4,5,5932,59.0,26000,0,0,1,6,3932.0,1844.0,87.7,0,0,127.0,294,1,1,0,1.0,NaN,1.0,NaN,0,4,6,4,4,5,8,12,6,10,0.0,0,0,5,94.1,75.0,0,0,66646,39316,15000,40646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,20000,20000,20000,36 months,7.84%,625.26,A,A4,Teacher,10+ years,MORTGAGE,48186.0,Not Verified,Sep-2018,Fully Paid,n,debt_consolidation,Debt consolidation,641xx,MO,24.68,0,Aug-1996,0,NaN,NaN,17,0,23822,58.8%,38,w,0.00,0.00,20784.182581,20784.18,20000.00,784.18,0.0,0.0,0.0,Apr-2019,17105.26,NaN,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,191246,0,9,0,2,16.0,14800,48.0,1,1,11800,54.0,40500,1,0,0,3,11250.0,13078.0,57.8,0,0,265.0,239,8,8,2,8.0,NaN,16.0,NaN,0,3,4,6,7,27,7,9,4,17,0.0,0,0,1,100.0,16.7,0,0,234182,38622,31000,30832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,15000,15000,15000,60 months,15.02%,357.01,C,C3,Salesperson,10+ years,RENT,70000.0,Not Verified,Sep-2018,Current,n,debt_consolidation,Debt consolidation,190xx,PA,18.05,0,Jul-1995,0,42.0,NaN,7,0,8112,66.5%,11,w,13398.12,13398.12,3188.060000,3188.06,1601.88,1586.18,0.0,0.0,0.0,Jun-2019,357.01,Jul-2019,Jun-2019,1,80.0,1,Individual,NaN,NaN,NaN,0,0,11619,0,1,0,0,34.0,3507,12.0,2,4,2561,28.0,12200,0,0,0,4,1660.0,3751.0

In [14]:
df2['term'] = df2['term'].str.strip(' months')
df2['term'] = df2['term'].astype(int)
df2.dtypes

loan_amnt                                       int64
funded_amnt                                     int64
funded_amnt_inv                                 int64
term                                            int64
int_rate                                       object
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
purpose                                        object
title                                          object
zip_code                    

In [0]:
def goodStanding(value):
  if value == "Current" or value == "Fully Paid":
    return(1);
  else: return(0);
  

In [16]:
df3 = df2.copy()
df3['loan_status_is_great']=df3['loan_status'].apply(goodStanding)

print(df3['loan_status'][50])
print(df3['loan_status_is_great'][50])

print('')

print(df3['loan_status'][49])
print(df3['loan_status_is_great'][49]) # 3 Examples to show it works

print('')

print(df3['loan_status'][1])
print(df3['loan_status_is_great'][1])


Charged Off
0

Current
1

Fully Paid
1


In [0]:
df4 = df3.copy()

df4['last_pymnt_d_year'] = pd.to_datetime(df4['last_pymnt_d'], infer_datetime_format=True)
df4['last_pymnt_d_month'] = pd.to_datetime(df4['last_pymnt_d'], infer_datetime_format=True)

In [0]:
df4['last_pymnt_d_year'] = df4['last_pymnt_d_year'].dt.year


In [0]:
df4['last_pymnt_d_month'] = df4['last_pymnt_d_month'].dt.month

In [20]:
finalDF = df4.copy()
finalDF.head(20)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status_is_great,last_pymnt_d_year,last_pymnt_d_month
0,25000,25000,25000,36,16.14%,880.66,C,C4,Business Planner,10+ years,RENT,82000.0,Source Verified,Sep-2018,Current,n,credit_card,Credit card refinancing,940xx,CA,19.60,0,Mar-1994,1,NaN,NaN,10,0,15437,59.4%,17,w,19828.26,19828.26,7881.110000,7881.11,5171.74,2709.37,0.0,0.0,0.0,Jun-2019,880.66,Jul-2019,Jun-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,39316,2,2,1,1,4.0,23879,59.0,4,5,5932,59.0,26000,0,0,1,6,3932.0,1844.0,87.7,0,0,127.0,294,1,1,0,1.0,NaN,1.0,NaN,0,4,6,4,4,5,8,12,6,10,0.0,0,0,5,94.1,75.0,0,0,66646,39316,15000,40646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1,2019.0,6.0
1,20000,20000,20000,36,7.84%,625.26,A,A4,Teacher,10+ years,MORTGAGE,48186.0,Not Verified,Sep-2018,Fully Paid,n,debt_consolidation,Debt consolidation,641xx,MO,24.68,0,Aug-1996,0,NaN,NaN,17,0,23822,58.8%,38,w,0.00,0.00,20784.182581,20784.18,20000.00,784.18,0.0,0.0,0.0,Apr-2019,17105.26,NaN,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,191246,0,9,0,2,16.0,14800,48.0,1,1,11800,54.0,40500,1,0,0,3,11250.0,13078.0,57.8,0,0,265.0,239,8,8,2,8.0,NaN,16.0,NaN,0,3,4,6,7,27,7,9,4,17,0.0,0,0,1,100.0,16.7,0,0,234182,38622,31000,30832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1,2019.0,4.0
2,15000,15000,15000,60,15.02%,357.01,C,C3,Salesperson,10+ years,RENT,70000.0,Not Verified,Sep-2018,Current,n,debt_consolidation,Debt consolidation,190xx,PA,18.05,0,Jul-1995,0,42.0,NaN,7,0,8112,66.5%,11,w,13398.12,13398.12,3188.060000,3188.06,1601.88,1586.18,0.0,0.0,0.0,Jun-2019,357.01,Jul-2019,Jun-2019,1,80.0,1,Individual,NaN,NaN,NaN,0,0,11619,0

# STRETCH OPTIONS

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01